# Formatação dos dados para Regionalização de Bacias usando Redes Neurais

## Importing Packages

In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")
sys.path.append("../database")

In [2]:
import pandas as pd
import numpy as np
#import tensorflow as tf
import os

In [3]:
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# # Plotting
# import plotly
# import plotly.graph_objs as go
# # import plotly.io as pio
# #import cufflinks as cf
# plotly.offline.init_notebook_mode(connected=True)


## Importing Data

In [4]:
caracFisicas = pd.read_excel("../database/dados.xls",sheet_name="Resultados")
caracFisicas = caracFisicas.drop(["SAT","PES","Nash","Nash Ver"],axis=1)

db = pd.ExcelFile("../database/dados.xls")
basins = db.sheet_names[6:]

In [5]:
caracUtilizadas = caracFisicas.drop(["Precipitação média - P (mm)","Capacidade de armazenamento do solo CAD (mm)",
                                    "Índice de compacidade da bacia - Kc",],axis=1)
caracUtilizadas.head()

,Estações ANA,Declividade - D (%),Comprimento do rio principal - CT (km),Área de contribuição da estação fluviométrica - A (km2),Perímetro da área de contribuição da estação fluviométrica - P (km),Comprimento total de drenagem - CTD (km),Densidade de drenagem - DD (km-1),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr
0,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.000000
1,34740000,0.064875,108123.318373,2221.988771,181.354250,356.001008,0.160217,59.941362,0.000014
2,34750000,0.056337,328854.100000,19185.920000,1256.998009,10051.126447,0.523880,73.945379,0.588056
3,35050000,0.092084,66700.567865,997.264362,218.297537,606.451358,0.608115,82.375460,0.534352
4,35125000,0.081096,96187.532702,1501.236654,261.225198,1501.190430,0.999969,84.314697,0.920865


In [6]:
caracFFN1 = caracUtilizadas.drop('Estações ANA',axis=1).columns
caracFFN2 = caracFFN1[[2,4,6,7]]
caracFFN3 = caracFFN2

## Formatting Data

In [7]:
def ShiftAndConcat(df,col,n):
    for i in range(n):
        if i ==0:
            temp = df[col]
        else:
            temp = pd.concat([temp,df[col].shift(-i)],axis=1)
            
    temp.columns = [col+"_%d" % (n-1-j) for j in range(n)]
    return temp


In [8]:
def FormatData(db,Plag,Elag,Qlag):
    i = max([Plag,Elag,Qlag])

    for k in range(len(basins)):
        PEQbasin = db.parse(basins[k])
        PEQbasin = PEQbasin.set_index("Data")
        Q = ShiftAndConcat(PEQbasin,"Q",i)
        P = ShiftAndConcat(PEQbasin,"P",i)
        E = ShiftAndConcat(PEQbasin,"E",i)
        dfBasin = pd.concat([E,P,Q],axis=1)
        colNames = dfBasin.columns
        for col in colNames:
            dfBasin = dfBasin.query("%s != -999" % col)
        dfBasin
        caracBasin = caracUtilizadas[caracUtilizadas["Estações ANA"] == int(basins[k])]
        caracBasinMatrix = np.ones((dfBasin.shape[0],caracBasin.shape[1])) * caracBasin.values
        caracBasinDf = pd.DataFrame(caracBasinMatrix,columns=caracBasin.columns)
        dfFormatted = pd.concat([caracBasinDf,dfBasin.reset_index().drop("Data",axis=1)],axis=1)
        if k == 0:
            dfCompleted = dfFormatted
        else:
            dfCompleted = pd.concat([dfCompleted, dfFormatted], axis=0)
    dfCompleted = dfCompleted.loc[dfCompleted.Q_0.notnull()]
    if Qlag < i:
        dfCompleted = dfCompleted.drop(["Q_%d" % (j) for j in range(Qlag,i)],axis=1)
    if Plag < i:
        dfCompleted = dfCompleted.drop(["P_%d" % (j) for j in range(Plag,i)],axis=1)
    if Elag < i:
        dfCompleted = dfCompleted.drop(["E_%d" % (j) for j in range(Elag,i)],axis=1)
        
    dfCompleted["Estações ANA"] = dfCompleted["Estações ANA"].astype(int).astype(str)
    return dfCompleted

In [9]:
dfCompleted = FormatData(db,3,1,3)
dfCompleted.head()

,Estações ANA,Declividade - D (%),Comprimento do rio principal - CT (km),Área de contribuição da estação fluviométrica - A (km2),Perímetro da área de contribuição da estação fluviométrica - P (km),Comprimento total de drenagem - CTD (km),Densidade de drenagem - DD (km-1),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr,E_0,P_2,P_1,P_0,Q_2,Q_1,Q_0
0,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,121.889904,204.2900,135.78000,142.92000,7.884968,6.138600,3.870129
1,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,129.330288,135.7800,142.92000,12.50800,6.138600,3.870129,2.136953
2,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,151.559616,142.9200,12.50800,3.03470,3.870129,2.136953,1.040503
3,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,174.559616,12.5080,3.03470,0.96898,2.136953,1.040503,0.629442
4,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.0,187.559616,3.0347,0.96898,1.95120,1.040503,0.629442,0.388273


In [10]:
caracDiffs = list(set(caracFFN1).difference(set(caracFFN2)))
caracDiffsLSTM = list(set(caracFFN1).difference(set(caracFFN2)))

In [11]:
dfFFN1 = FormatData(db,3,1,3)
dfLSTM = FormatData(db,6,1,1).drop(caracDiffs,axis=1)

In [12]:
maxs = dfFFN1.iloc[:,1:-1].max().values
mins = dfFFN1.iloc[:,1:-1].min().values
maxLSTM = dfLSTM.iloc[:,1:-1].max().values
minLSTM = dfLSTM.iloc[:,1:-1].min().values

In [13]:
dfFFN1norm = dfFFN1.copy()
dfLSTMnorm = dfLSTM.copy()
dfFFN1norm.iloc[:,1:-1] = dfFFN1.iloc[:,1:-1].apply(lambda x: (x-mins)/(maxs-mins),axis=1)
dfLSTMnorm.iloc[:,1:-1] = dfLSTM.iloc[:,1:-1].apply(lambda x: (x-minLSTM)/(maxLSTM-minLSTM),axis=1)

In [14]:
dfFFN2norm = dfFFN1norm.drop(caracDiffs,axis=1)
dfFFN3norm = dfFFN1norm.drop(caracDiffs+["Q_2","Q_1"],axis=1)

In [15]:
dfLSTMnorm.head()

,Estações ANA,Área de contribuição da estação fluviométrica - A (km2),Comprimento total de drenagem - CTD (km),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr,E_0,P_5,P_4,P_3,P_2,P_1,P_0,Q_0
0,34730000,0.019908,0.011994,0.0,0.0,0.767287,0.333448,0.221624,0.233278,0.020416,0.004953,0.001582,0.629442
1,34730000,0.019908,0.011994,0.0,0.0,0.863514,0.221624,0.233278,0.020416,0.004953,0.001582,0.003185,0.388273
2,34730000,0.019908,0.011994,0.0,0.0,0.971218,0.233278,0.020416,0.004953,0.001582,0.003185,0.000546,0.152000
3,34730000,0.019908,0.011994,0.0,0.0,0.910304,0.020416,0.004953,0.001582,0.003185,0.000546,0.011401,0.143093
4,34730000,0.019908,0.011994,0.0,0.0,0.905346,0.004953,0.001582,0.003185,0.000546,0.011401,0.071503,0.627194


In [16]:
dfCompleted = dfCompleted.reset_index(drop=True)#.drop(caracDiffs,axis=1)


In [17]:
dfCompletedNorm = dfCompleted.copy()
dfCompletedNorm.iloc[:,1:-1] = dfCompleted.iloc[:,1:-1].apply(lambda x: (x-mins)/(maxs-mins),axis=1)


In [35]:
maxminFFN = pd.DataFrame([maxs,mins],columns=dfFFN1.columns[1:-1])
maxminFFN.drop(caracDiffs+["Q_2","Q_1"],axis=1).to_csv("../database/MaxMinFFN3.csv",index=False)

In [34]:
caracDiffs + ["E_0","P_2","P_1","P_0","P_0","Q_0"]
maxminFFN.drop(caracDiffs+["Q_2","Q_1"],axis=1)

,Área de contribuição da estação fluviométrica - A (km2),Comprimento total de drenagem - CTD (km),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr,E_0,P_2,P_1,P_0
0,20664.322313,14792.265799,85.743443,0.988209,205.998469,612.66,612.66,612.66
1,495.858575,355.643834,56.773914,0.000000,70.901645,0.00,0.00,0.00


## Saving Data

In [18]:
dfCompletedNorm.to_csv("../database/DadosFFN1.csv",index=False)
dfCompletedNorm.drop(caracDiffs,axis=1).to_csv("../database/DadosFFN2.csv",index=False)
dfCompletedNorm.drop(caracDiffs2,axis=1).to_csv("../database/DadosFFN3.csv",index=False)

NameError: name 'caracDiffs2' is not defined

In [ ]:
dfLSTMnorm.to_csv("../database/DadosLSRMrg.csv",index=False)